## Quarto - a framework for reproducible research

Quarto enables you to weave together content and executable code into a finished document. To learn more about Quarto see <https://quarto.org>. When you click the **Render** button a document will be generated that includes both content and the output of embedded code. The template below can be used to submit your answers for the mandatory assignment. Within the template you find all typical issues. By default, code is omitted in the output (can be changed with `echo: true` in line 6). Formatting is done using markdown (e.g., check out how to use \# to define sections and subsections or how to generate bold text). Footnotes and citations are easy. You can also include math notation using LaTeX style. The template contains inline code execution (e.g. to automatically return the ticker of the asset with the highest Sharpe ratio) and shows how to embed figures into the document.

## Exercise

1. Download daily adjusted prices for all constituents of three stocks: AAPL, MSFT, and BA for the period from January 1st, 2000 until December 31st, 2022 from Yahoo!Finance. Explicitly include the code you use to download the data into your report. Compute the daily returns for each of the tickers.

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf

ticker = ["AAPL", "MSFT", "BA"]
prices = (yf.download(tickers=ticker,
                            start="2000-01-01",
                            end="2022-12-31",
                            progress=False)
  .melt(ignore_index=False,
        var_name=["variable", "symbol"])
  .reset_index()
  .pivot(index=["Date", "symbol"],
         columns="variable",
         values="value")
  .get("Adj Close")
  .reset_index()
  .rename(columns = {"Date": "date",
                     "Adj Close": "adjusted"})
)

returns = (prices
  .assign(ret = lambda x: x.groupby("symbol")["adjusted"].pct_change())
  .get(["symbol", "date", "ret"])
  .dropna(subset="ret")
)

1. Visualize the distribution of the daily returns.


In [ ]:
#| fig-cap: "Distribution of daily stock returns."
from plotnine import *
from mizani.formatters import percent_format

returns_figure = (
  ggplot(returns, aes(x="ret", fill="symbol")) +
  geom_histogram(bins=100) +
  labs(x="", y="",
       title="Distribution of daily stock returns",
       fill ="") +
  facet_wrap("symbol") +
  scale_x_continuous(labels=percent_format())
)
returns_figure.draw()

1. Compute the sample mean $\mu$ and the variance-covariance matrix $\Sigma$ of the returns.
Which of the individual assets delivered the highest Sharpe ratio (assume the risk-free rate is zero) during the sample period?^[Whenever reporting performance measures, compute annualized values and clearly state how you calculated the measure.]
The estimated moments may later be used for the construction of optimal portfolios (@Markowitz1952; @Merton1972)


In [ ]:
returns_matrix = (prices
  .pivot(columns="symbol", 
         values="adjusted", 
         index="date")
  .pct_change()
  .dropna()
)

mu = np.array(returns_matrix.mean()).T
sigma = np.array(returns_matrix.cov())

sharpe_ratio = pd.DataFrame({"ticker": ticker,
  "Sharpe": np.sqrt(250) * mu / np.sqrt(np.diag(sigma))})

**Answer: ** The asset with the highest annualized Sharpe-ratio (computed as $\sqrt{250}\frac{\hat\mu_i}{\hat\sigma_i}$, where $\hat\mu_i$ is the sample mean of asset $i$s dailies returns and $\hat\sigma_i$is the standard deviation of asset $i$s dailys returns) is `{python} sharpe_ratio['ticker'][sharpe_ratio['Sharpe'].idxmax()]` with an annualized Sharpe-ratio of `{python} round(sharpe_ratio['Sharpe'].max(), 3)`. A table with all Sharpe ratios can be found below. 

In [ ]:
#| tbl-cap: "Historical annualized Sharpe-ratios"
sharpe_ratio

## How to generate a script to submit for peer feedback?

This is easy. Once you are done with working on your quarto file, you simply run the line of code below (in a separate session, not as part of rendering the quarto file itself). Once executed you will see a new R-script which contains only the code.

In [ ]:
#| echo: true
#| eval: false
knitr::purl("MA_template.qmd")